In [1]:
import sys
import yaml

import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from botorch.models.transforms.input import AffineInputTransform

from lume_model.utils import variables_from_yaml
from lume_model.torch import LUMEModule, PyTorchModel

sys.path.append("../")
from utils import update_input_variables_to_transformer

## Load Uncalibrated LUME-model

In [2]:
model_dir = "../lcls_cu_injector_nn_model/"

# create base model
base_model = torch.load(model_dir + "model/model.pt")

# load sim_to_nn transformers
input_sim_to_nn = torch.load(model_dir + "model/input_sim_to_nn.pt")
output_sim_to_nn = torch.load(model_dir + "model/output_sim_to_nn.pt")

# load pv_to_sim transformers
input_pv_to_sim = torch.load(model_dir + "model/input_pv_to_sim.pt")
output_pv_to_sim = torch.load(model_dir + "model/output_pv_to_sim.pt")

# load in- and output variable specification
input_variables, output_variables = variables_from_yaml(open(model_dir + "model/pv_variables.yml"))

# create LUME-model
lume_model = PyTorchModel(
    model_file=model_dir + "model/model.pt",
    input_variables=input_variables,
    output_variables=output_variables,
    input_transformers=[input_pv_to_sim, input_sim_to_nn],
    output_transformers=[output_sim_to_nn, output_pv_to_sim],
)

# wrap in LUMEModule
lume_module = LUMEModule(
    model=lume_model,
    feature_order=lume_model.features,
    output_order=lume_model.outputs[:2],
)

## Load Calibrated LUME-model

In [3]:
reg = "high"  # "low" or "high"

# create base model
base_model = torch.load(model_dir + "model/model.pt")

# load nn_to_cal transformers
input_nn_to_cal = torch.load(f"input_nn_to_cal_{reg}_reg.pt")
output_nn_to_cal = torch.load(f"output_nn_to_cal_{reg}_reg.pt")

# load sim_to_nn transformers
input_sim_to_nn = torch.load(model_dir + "model/input_sim_to_nn.pt")
output_sim_to_nn = torch.load(model_dir + "model/output_sim_to_nn.pt")

# load pv_to_sim transformers
input_pv_to_sim = torch.load(model_dir + "model/input_pv_to_sim.pt")
output_pv_to_sim = torch.load(model_dir + "model/output_pv_to_sim.pt")

# load in- and output variable specification
input_variables, output_variables = variables_from_yaml(open(model_dir + "model/pv_variables.yml"))

# create LUME-model
cal_lume_model = PyTorchModel(
    model_file=model_dir + "model/model.pt",
    input_variables=input_variables,
    output_variables=output_variables,
    input_transformers=[input_pv_to_sim, input_sim_to_nn, input_nn_to_cal],
    output_transformers=[output_nn_to_cal, output_sim_to_nn, output_pv_to_sim],
)

# wrap in LUMEModule
cal_lume_module = LUMEModule(
    model=cal_lume_model,
    feature_order=cal_lume_model.features,
    output_order=cal_lume_model.outputs[:2],
)

## Uncalibrated Variable Ranges and Defaults

In [4]:
uncalibrated_variables = pd.DataFrame(columns=["Variable", "Min", "Max", "Default"])
for i, k in enumerate(input_variables.keys()):
    v = input_variables[k]
    uncalibrated_variables.loc[len(uncalibrated_variables.index)] = [k, v.value_range[0], v.value_range[1], v.default]
uncalibrated_variables

,Variable,Min,Max,Default
0,CAMR:IN20:186:R_DIST,210.212478,499.999608,423.867825
1,Pulse_length,1.818182,7.271860,1.855051
2,FBCK:BCI0:1:CHRG_S,0.250000,0.250000,0.250000
3,SOLN:IN20:121:BACT,0.377408,0.498380,0.477969
4,QUAD:IN20:121:BACT,-0.020984,0.020999,-0.001499
5,QUAD:IN20:122:BACT,-0.020999,0.020999,-0.000687
6,ACCL:IN20:300:L0A_ADES,58.000000,58.000000,58.000000
7,ACCL:IN20:300:L0A_PDES,-24.998715,9.991752,-9.535973
8,ACCL:IN20:400:L0B_ADES,70.000000,70.000000,70.000000
9,ACCL:IN20:400:L0B_PDES,-24.999726,9.998905,9.855662


## Updated Variable Ranges and Defaults

In [5]:
updated_input_variables = update_input_variables_to_transformer(cal_lume_model, 2)
updated_variables = pd.DataFrame(columns=["Variable", "Min", "Max", "Default"])
for i, k in enumerate(updated_input_variables.keys()):
    v = updated_input_variables[k]
    updated_variables.loc[len(updated_variables.index)] = [k, v.value_range[0], v.value_range[1], v.default]
updated_variables

,Variable,Min,Max,Default
0,CAMR:IN20:186:R_DIST,242.167102,540.235435,461.928045
1,Pulse_length,-1.149606,3.732711,-1.116599
2,FBCK:BCI0:1:CHRG_S,0.250018,0.250018,0.250018
3,SOLN:IN20:121:BACT,0.413516,0.504213,0.488911
4,QUAD:IN20:121:BACT,-0.002711,0.033279,0.013993
5,QUAD:IN20:122:BACT,-0.027210,0.022778,-0.003034
6,ACCL:IN20:300:L0A_ADES,58.000000,58.000000,58.000000
7,ACCL:IN20:300:L0A_PDES,-12.482394,22.978195,3.188100
8,ACCL:IN20:400:L0B_ADES,70.000000,70.000000,70.000000
9,ACCL:IN20:400:L0B_PDES,-2.255725,36.273309,36.115617


In [6]:
pd.concat([uncalibrated_variables, updated_variables], axis=1)

,Variable,Min,Max,Default,Variable,Min,Max,Default
0,CAMR:IN20:186:R_DIST,210.212478,499.999608,423.867825,CAMR:IN20:186:R_DIST,242.167102,540.235435,461.928045
1,Pulse_length,1.818182,7.271860,1.855051,Pulse_length,-1.149606,3.732711,-1.116599
2,FBCK:BCI0:1:CHRG_S,0.250000,0.250000,0.250000,FBCK:BCI0:1:CHRG_S,0.250018,0.250018,0.250018
3,SOLN:IN20:121:BACT,0.377408,0.498380,0.477969,SOLN:IN20:121:BACT,0.413516,0.504213,0.488911
4,QUAD:IN20:121:BACT,-0.020984,0.020999,-0.001499,QUAD:IN20:121:BACT,-0.002711,0.033279,0.013993
5,QUAD:IN20:122:BACT,-0.020999,0.020999,-0.000687,QUAD:IN20:122:BACT,-0.027210,0.022778,-0.003034
6,ACCL:IN20:300:L0A_ADES,58.000000,58.000000,58.000000,ACCL:IN20:300:L0A_ADES,58.000000,58.000000,58.000000
7,ACCL:IN20:300:L0A_PDES,-24.998715,9.991752,-9.535973,ACCL:IN20:300:L0A_PDES,-12.482394,22.978195,3.188100
8,ACCL:IN20:400:L0B_ADES,70.000000,70.000000,70.000000,ACCL:IN20:400:L0B_ADES,70.000000,70.000000,70.000000
9,ACCL:IN20:400:L0B_PDES,-24.999726,9.998905,9.855662,ACCL:IN20:400:L0B_PDES,-2.255725,36.273309,36.115617


In [7]:
# double check new ranges
x_old = torch.from_numpy(uncalibrated_variables["Min"].values)
for transformer in lume_model.input_transformers:
    x_old = transformer(x_old)

# forward pass with calibration
x_new = torch.from_numpy(updated_variables["Min"].values)
for transformer in cal_lume_model.input_transformers:
    x_new = transformer(x_new)

# check values match before passing NN model
torch.allclose(x_old, x_new)

True

## Create Variables YAML-File

In [8]:
# load old config from file
config = yaml.safe_load(open(model_dir + "model/pv_variables.yml"))

# update input_variables in config
for key in config["input_variables"].keys():
    config["input_variables"][key]["range"] = [
        float(updated_variables.loc[updated_variables["Variable"] == key]["Min"].values[0]),
        float(updated_variables.loc[updated_variables["Variable"] == key]["Max"].values[0]),
    ]
    config["input_variables"][key]["default"] = float(updated_variables.loc[updated_variables["Variable"] == key]["Default"].values[0])

# write to file
with open(f"pv_variables_{reg}_reg.yml", "w") as f:
    f.write(
        yaml.dump({k: v for k, v in config.items() if k == "input_variables"}, default_flow_style=None, sort_keys=False) + 
        yaml.dump({k: v for k, v in config.items() if k == "output_variables"}, default_flow_style=False, sort_keys=False)
    )
    f.close()